# Demo (Colab) — HW3 TTS Vocoder\nМинимальный демо-ноутбук для проверки инференса.

## 1) Клонирование / распаковка проекта\nЕсли используешь zip с Google Drive — распакуй в рабочую директорию Colab.

In [ ]:
!pip install -q -r requirements.txt

## 2) Скачать/подготовить чекпоинт\nИспользуй `scripts/download_checkpoints.py` или загрузи `best_generator.pt`.

In [ ]:
# !python scripts/download_checkpoints.py --url <DIRECT_URL> --out checkpoints/best_generator.pt

## 3) Подготовить пример датасета

In [ ]:
!python scripts/make_dummy_custom_dir_dataset.py --out examples/custom_dir_dataset_example

## 4) Запустить synthesize.py (resynthesis)

In [ ]:
# Команда ниже заработает после того, как у тебя будет обученный чекпоинт\n# !python synthesize.py infer.checkpoint_path=checkpoints/best_generator.pt infer.input_dir=examples/custom_dir_dataset_example infer.output_dir=outputs/resynthesis

## 5) Прослушать файлы

In [ ]:
from IPython.display import Audio\nimport os\nout_dir = 'outputs/resynthesis'\nif os.path.exists(out_dir):\n    files = sorted([f for f in os.listdir(out_dir) if f.endswith('.wav')])\n    print(files)\n    if files:\n        display(Audio(os.path.join(out_dir, files[0])))\nelse:\n    print('Сначала запусти synthesize.py')